In [1]:
import google.generativeai as genai
import os
import json
from openai import OpenAI
import tqdm
import hashlib
import time
import threading

/home/rrenaud/miniconda3/envs/med_bib/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
openai_key = open(os.path.expanduser('~/.openai_key.txt')).read().strip()
client = OpenAI(api_key=openai_key)

In [9]:
stories = json.load(open('spanish_stories/stories_00.json', 'r'))

In [8]:
print(stories[0]['story'])

Un día, un niño llamado Leo jugó con cartas. Él tomó las cartas y las puso en un montón. Después, él las puso en otro montón. ¡Él inventó un nuevo juego! 

Leo miró las cartas. Él vio un gato. "¡Miau!" dijo el gato. Después, él vio un perro. "¡Guau!" dijo el perro. Leo sonrió. Él pensó que su juego era divertido. 

Después de jugar, Leo se fue a la cama. Él soñaba con el gato y el perro. En su sueño, el gato y el perro corrieron juntos. Ellos eran felices.

¡De repente! Leo despertó. Él sintió que algo no estaba bien. Él fue a la ventana. Él vio al gato y al perro jugando afuera. ¡El sueño se había hecho realidad! 

Leo corrió afuera y jugó con el gato y el perro. Él jugó despacio. Él no quería asustarlos. Él era feliz. Él sabía que su sueño había sido real. 



In [10]:
def make_translation_request(story):
    prompt = f"""You are tasked with translating a children's story from Spanish to English. 
Your goal is to create an accurate translation that maintains the original story's tone, style, and 
meaning while making it accessible to English-speaking children who only know simple words. 
Here is the Spanish story <spanish_story>{story}</spanish_story> 
Present your translation within <english_translation> tags."""
    return prompt
    

In [11]:
print(make_translation_request(stories[0]['story']))

You are tasked with translating a children's story from Spanish to English. 
Your goal is to create an accurate translation that maintains the original story's tone, style, and 
meaning while making it accessible to English-speaking children who only know simple words. 
Here is the Spanish story <spanish_story>Un día, un niño llamado Leo jugó con cartas. Él tomó las cartas y las puso en un montón. Después, él las puso en otro montón. ¡Él inventó un nuevo juego! 

Leo miró las cartas. Él vio un gato. "¡Miau!" dijo el gato. Después, él vio un perro. "¡Guau!" dijo el perro. Leo sonrió. Él pensó que su juego era divertido. 

Después de jugar, Leo se fue a la cama. Él soñaba con el gato y el perro. En su sueño, el gato y el perro corrieron juntos. Ellos eran felices.

¡De repente! Leo despertó. Él sintió que algo no estaba bien. Él fue a la ventana. Él vio al gato y al perro jugando afuera. ¡El sueño se había hecho realidad! 

Leo corrió afuera y jugó con el gato y el perro. Él jugó despaci

In [18]:
def execute_translation_request(story_text):
    req = make_translation_request(story_text)
    return client.chat.completions.create(
        model="gpt-4o-mini",
        max_tokens=4096,
        messages = [{"role": "user", "content": req}])

def get_translation(response):
    return response.choices[0].message.content.split('<english_translation>')[1].split('</english_translation>')[0]

def read_hashes():
    hashes = set()
    for fn in os.listdir('spanish_stories_translated'):
        split_fn = fn.split('-')
        hash = split_fn[2].removesuffix('.json')
        hashes.add(hash)
    return hashes


In [15]:
def translate_and_save(story_json, thread_id):
    resp = execute_translation_request(story_json['story'])
    translation = get_translation(resp)
    md5 = hashlib.md5(story_json['story'].encode()).hexdigest()
    story_json['translation'] = translation
    with open(f'spanish_stories_translated/story-{thread_id}-{md5}.json', 'w') as f:
        json.dump(story_json, f)

def translate_story_range(start, end, thread_id):
    successes = 0
    fails = 0
    skips = 0
    for i in range(start, end):
        if hashlib.md5(stories[i]['story'].encode()).hexdigest() in hashes:
            skips += 1
            continue
        try:
            translate_and_save(stories[i], thread_id)
            successes += 1
        except Exception as e:
            print('failed story', i, 'on thread', thread_id, 'because', e)
            time.sleep(30)
            fails += 1
        if i % 10 == 0:
            print('thread', thread_id, 'completed', i - start, 'stories', 'successes:', successes, 'fails:', fails, 'skips:', skips)

In [19]:
hashes = read_hashes()
print('read hashes', len(hashes), 'examples', list(hashes)[:5])

NUM_THREADS = 1
NUM_STORIES = len(stories)

stories_per_thread = NUM_STORIES // NUM_THREADS
start_of_ranges = [i * stories_per_thread for i in range(NUM_THREADS)]
end_of_ranges = [(i + 1) * stories_per_thread for i in range(NUM_THREADS)]

threads = []

for thread_id, (start, end) in enumerate(zip(start_of_ranges, end_of_ranges)):
    t = threading.Thread(target=translate_story_range, args=(start, end, thread_id))
    threads.append(t)
    t.start()

for t in threads:
    t.join()


read hashes 99994 examples ['755725a83fab4b98e52686da6008cd3d', '1512b7c811dd088f592fba2c63cc539b', 'a0e3dbe50394806d351fa1c03a7f924e', '2dccb68168dc8d63addf310a373f8366', '1b74df8e424a35e91d32473fc7eb4f9b']
